# TIPE 2023-2024 "Jeux et sports" 
### MORAIN Arthur MP2 Montaigne

## <ins>Optimisation de l'arrêt au stand en Formule 1</ins> 

### **<ins>V- Construction de notre base de travail :</ins>**   

-> L'objectif ici est de construire notre base de données finale, qui nous permettra ensuite de mettre en place notre modèle de pneumatiques et puis nos deux expériences.

#### **<ins>a- Le Chemin, le Dataset, les Modules :</ins>**  

encore

In [7]:
path = '/Users/morainarthur/Home/Datascience/redbull-pit-strategy/notebooks'
data_path = '/Users/morainarthur/Home/Datascience/redbull-pit-strategy/data/'

In [2]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

#### **<ins>b- Données météo :</ins>**  

In [8]:
file = open(data_path+'weathers_data.pickle','rb')
df_weather = pickle.load(file)
df_weather['eventYear'] = pd.to_datetime(df_weather['EventDate'], utc=True).dt.year

#### **<ins>c- Données des résultats :</ins>**  

In [9]:
file = open(data_path+'results_data.pickle','rb')
df_res = pickle.load(file)

df_res['eventYear'] = pd.to_datetime(df_res['EventDate'], utc=True).dt.year
df_res['TimeSec'] = df_res['Time']/np.timedelta64(1, 's')


## on garde seulement ce qui concerne la course
df_res = df_res[(df_res['session']=='Race')]
print('-'*100,'\n', 'df_res size is: ', df_res.shape,'\n','-'*100,'\n')
print()

df_res= df_res[['Position','GridPosition','FullName','RoundNumber','Abbreviation',
                'EventName','eventYear','session','Time','TimeSec','TeamName', 'EventDate']]

---------------------------------------------------------------------------------------------------- 
 df_res size is:  (2060, 29) 
 ---------------------------------------------------------------------------------------------------- 




#### **<ins>d-  Les données de temps :</ins>**  

On charge le dataset puis on convertit les temps en seconde, plus agréable à manipuler

In [10]:
file = open(data_path+'laps_data.pickle','rb')
df_laps = pickle.load(file)

In [11]:
df_laps.shape

(285769, 36)

In [12]:
df_laps.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'Team', 'LapStartTime',
       'LapStartDate', 'TrackStatus', 'Position', 'Deleted', 'DeletedReason',
       'FastF1Generated', 'IsAccurate', 'RoundNumber', 'EventName', 'country',
       'session', 'EventDate'],
      dtype='object')

In [14]:
drops = ['SpeedI1', 'SpeedI2', 'SpeedFL']
df_laps = df_laps.drop(drops,axis=1).reset_index(drop=True)

In [15]:
df_laps.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre',
       'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position',
       'Deleted', 'DeletedReason', 'FastF1Generated', 'IsAccurate',
       'RoundNumber', 'EventName', 'country', 'session', 'EventDate'],
      dtype='object')

In [16]:
df_laps['eventYear'] = pd.to_datetime(df_laps['EventDate'], utc=True).dt.year
df_laps['lapTimeSec'] = df_laps['LapTime']/np.timedelta64(1, 's')

#### **<ins>e- Fusion des données météo et de temps :</ins>**  

In [17]:
df_weather = df_weather.sort_values( ['EventName','eventYear','session', 'Time']).reset_index(drop=True)
df_laps = df_laps.sort_values( ['EventName','eventYear','session','Team','Driver', 'Time']).reset_index(drop=True)

In [18]:
df_laps.columns

Index(['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre',
       'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position',
       'Deleted', 'DeletedReason', 'FastF1Generated', 'IsAccurate',
       'RoundNumber', 'EventName', 'country', 'session', 'EventDate',
       'eventYear', 'lapTimeSec'],
      dtype='object')

In [19]:
dflaps_columns = ['Time', 'Driver', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint',
       'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre',
       'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position',
       'Deleted', 'DeletedReason', 'FastF1Generated', 'IsAccurate',
       'RoundNumber', 'EventName', 'country', 'session', 'EventDate',
       'eventYear', 'lapTimeSec']

In [21]:
%%time

import warnings # permettra de terminer le programme malgré un avertissement
warnings.filterwarnings("ignore")

# group lap data by event, session... etc and then go to weather data and extract relevant weather information.
# since time doesn't quite match, we find the closest possible time between both tables.
# merge_asof allows us to find the nearest time between the two tables and combine them.
def func_merger_lap_weather(d_f):
    '''
    func_merger_lap_weather method merges weather and laps data
    it returns all columns after merge
    '''    
    
    d_f = d_f.sort_values('Time').reset_index(drop=True)
    
    EventName = np.unique(d_f['EventName'])[0]
    session = np.unique(d_f['session'])[0]
    eventYear = np.unique(d_f['eventYear'])[0]
    Driver = np.unique(d_f['Driver'])
    
    sub_weather = df_weather[(df_weather['EventName']==EventName) &
                            (df_weather['session']==session) &
                            (df_weather['eventYear']==eventYear) ].sort_values('Time').reset_index(drop=True)
    
    sub_result = df_res[(df_weather['EventName']==EventName) &
                            (df_weather['session']==session) &
                            (df_weather['eventYear']==eventYear) ].sort_values('Time').reset_index(drop=True)
    
    d_f_merged = pd.merge_asof(d_f, sub_weather, 
                          on="Time", direction='nearest',
                         suffixes=('', '_drop'))
    d_f_merged.drop([col for col in d_f_merged.columns if 'drop' in col], axis=1, inplace=True)
    
#     print(d_f.shape,Driver, len(d_f_merged), EventName , eventYear, np.unique(d_f['Team']), 
#           session, len(df_weather))
    weather_cols = ['WindSpeed', 'AirTemp','Humidity', 'Pressure','TrackTemp','Rainfall','WindDirection']
    laps_cols = ['Time', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime', 'RoundNumber',
       'PitInTime', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife','FreshTyre', 'LapStartTime', 'TrackStatus',
       'LapStartDate',  'country', 'EventDate','lapTimeSec','Sector1Time','Sector2Time','Sector3Time',
             'Sector1SessionTime','Sector2SessionTime','Sector3SessionTime', 'IsAccurate']
    laps_cols.extend(weather_cols) 
    
    return  d_f_merged[laps_cols]

df_lapWeather = df_laps.groupby(['EventName', 'eventYear','Team','session','Driver'])[dflaps_columns].apply(func_merger_lap_weather).reset_index()


CPU times: user 8min 37s, sys: 2.55 s, total: 8min 39s
Wall time: 8min 40s


In [24]:
df_lapWeather.columns

Index(['EventName', 'eventYear', 'Team', 'session', 'Driver', 'level_5',
       'Time', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime',
       'RoundNumber', 'PitInTime', 'SpeedST', 'IsPersonalBest', 'Compound',
       'TyreLife', 'FreshTyre', 'LapStartTime', 'TrackStatus', 'LapStartDate',
       'country', 'EventDate', 'lapTimeSec', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'IsAccurate', 'WindSpeed', 'AirTemp', 'Humidity',
       'Pressure', 'TrackTemp', 'Rainfall', 'WindDirection'],
      dtype='object')

In [25]:
df_lapWeather.head(5)

,EventName,eventYear,Team,session,Driver,level_5,Time,DriverNumber,LapTime,LapNumber,...,Sector2SessionTime,Sector3SessionTime,IsAccurate,WindSpeed,AirTemp,Humidity,Pressure,TrackTemp,Rainfall,WindDirection
0,70th Anniversary Grand Prix,2020,Alfa Romeo Racing,FP1,KUB,0,0 days 00:18:07.531000,88,NaT,1.0,...,0 days 00:17:36.888000,0 days 00:18:07.598000,False,2.7,25.3,49.6,1001.5,34.6,False,231.0
1,70th Anniversary Grand Prix,2020,Alfa Romeo Racing,FP1,KUB,1,0 days 00:35:28.649000,88,NaT,2.0,...,0 days 00:34:57.975000,0 days 00:35:28.775000,False,3.4,26.4,43.6,1001.5,37.2,False,222.0
2,70th Anniversary Grand Prix,2020,Alfa Romeo Racing,FP1,KUB,2,0 days 00:37:00.685000,88,0 days 00:01:32.036000,3.0,...,0 days 00:36:35.547000,0 days 00:37:00.685000,True,2.1,26.0,44.8,1001.5,36.7,False,212.0
3,70th Anniversary Grand Prix,2020,Alfa Romeo Racing,FP1,KUB,3,0 days 00:39:11.032000,88,0 days 00:02:10.347000,4.0,...,0 days 00:38:34.621000,0 days 00:39:11.032000,True,2.5,26.2,43.9,1001.5,37.6,False,189.0
4,70th Anniversary Grand Prix,2020,Alfa Romeo Racing,FP1,KUB,4,0 days 00:40:41.364000,88,0 days 00:01:30.332000,5.0,...,0 days 00:40:16.516000,0 days 00:40:41.364000,True,1.2,26.3,43.3,1001.5,37.9,False,219.0


L'objectif est atteint, plus qu'à y travailler

#### **<ins>f- Conversion et enregistrement :</ins>**  

In [34]:
type(df_lapWeather)

pandas.core.frame.DataFrame

In [27]:
csv_data = df_lapWeather.to_csv()

In [28]:
df_lapWeather.shape

(285769, 39)

In [39]:
df_lapWeather.to_csv('/Users/morainarthur/Home/Datascience/redbull-pit-strategy/data/lapweatherf1.csv', index = False ) 

### **<ins>Remarque :</ins>**   

-> Ici se termine un premier quart du projet

-> On a obtenu une base de données avec laquelle nous allons pouvoir travailler facilement

-> Je ne pensais pas que cette étape serait aussi conséquente

31/10/2023